In [ ]:
install.packages("devtools")
devtools::install_github("hadley/devtools") ## for latest version
devtools::install_github("rstudio/sparklyr")
devtools::install_github("tidyverse/dplyr")


In [ ]:
pip install --upgrade tornado==5.1.1

In [ ]:
library(SparkR)

In [ ]:
sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g")) 

In [ ]:
#Crearemos el esquema de nuestros datos 
schema_departure <- structType(structField("date","int"),
                    structField("delay","int"),
                    structField("distance","int"),
                    structField("origin","string"),
                    structField("destination","string"))
# Realizamos la lectura de nuestro datasets indicando una serie de parametros como el formato de los datos, la cabecera o el schema a seguir.
df_departure <- read.df("data/departuredelays.csv", source = "csv", header="true", schema= schema_departure)


## Mostrar los dataframes
- head
- show
- showDF
- take


In [ ]:
#Por defecto muestra las 6 primeras líenas
head(df_departure)

In [ ]:
show(df_departure)

In [ ]:
# Por defecto muestra las 20 primeras filas
# Vertical nos permite monstrar un dataframe por registro siendo las en este caso las filas el nombre de nuestras columnas
showDF(df_departure,numRows = 15, truncate = TRUE, vertical = FALSE)

In [ ]:
# Es obligatorio indicar el numero de filas que quiere que te muestré
take(df_departure, 5)

## Primera exploración de los datos:
- summary
- describe
- summarize
- printSchema
- str 
- dtypes


In [ ]:
# Nos arrojará una tabla indicando las principales medidas estadisticas de nuestro datasets
showDF(summary(df_departure))

In [ ]:
# Al igual que la anterior formula describe nos muestra tambien medidas estadisticas pero sin los percentiles, 
showDF(describe(df_departure))

In [ ]:
head(summarize(groupBy(df_departure,df_departure$origin), avg = avg(df_departure$delay)))

In [ ]:
# Muestra la estructura de un SparkDataFrame, incluidos los nombres de columna, los tipos de columna, así como una pequeña muestra de filas.
str(df_departure)

In [ ]:
# Devuelve todos los nombres de columna y sus tipos de datos como una lista
dtypes(df_departure)

In [ ]:
# Imprimimos el esquema de nuestro conjunto de datos
printSchema(df_departure)

## SparkSQL

- createOrReplaceTempView
- sql

In [ ]:
# Crearemos una vista temporal para poder utilizar el lenguaje SQL
createOrReplaceTempView(df_departure,"us_delay_flights_tbl")


### Filtra los vuelos con mas de 1000 km

In [ ]:
# Monstrarmos el origen, destino y distancia para los vuelos mas de 1000
df_mas_1000_distance <- sql("SELECT distance, origin, destination
                               FROM us_delay_flights_tbl WHERE distance > 1000
                               ORDER BY distance DESC")
showDF(df_menos_1000_distance)

### Categoriza los vuelos segun el delays

In [ ]:
# Utilizaremos la clausula Case para categorizar los vuelos segun el tiempo del delay
df_delay <- sql("SELECT delay, origin, destination,
                  CASE
                    WHEN delay > 360 THEN 'Very Long Delays'
                    WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
                    WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
                    WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                    WHEN delay = 0 THEN 'No Delays'
                    ELSE 'Early'
                  END AS Flight_Delays
                  FROM us_delay_flights_tbl
                  ORDER BY origin, delay DESC")
showDF(df_delay)

## Manipulacion de los datos
- select
- filter
- orderBy
- distinct

In [ ]:
# Usaremos las diferentes API de R para seleccionar los vuelos con mas de 1000 de distancia

# Podemos indicar las columnas mediante "nombre_columna" o datasets$nombre_columna

temp <- filter(df_departure,"distance > 1000")
temp <- select(temp,"distance" , "origin", "destination")
temp <- orderBy(temp, desc(temp$distance))
flights_distance_plus_1000 <- distinct(temp)
showDF(flights_distance_plus_1000)

### Número de destinos diferentes por origen


In [ ]:

Destinos <- agg(groupBy(df_departure, "origin"), numero_destinos = countDistinct(df_departure$destination))

showDF(orderBy(Destinos,desc(Destinos$numero_destinos)), 10)

## Guardar los datos

In [ ]:
# Guardaremos nuestros datos en formato parquet mediante la funcion write.df o write.parquet
# De forma similar podremos guardar el datasets en los diferentes formatos soportados por Spark R (Json,CSV,TXT,Parquet, ORC, jbdc), liberías externas (Avro)
write.df(df_departure, path = "data/departuredelays.parquet", source = "parquet", mode = "overwrite")


#Cargamos los datos de parquet
df_more_1000_distance <- read.df("data/departuredelays.parquet", source = "parquet", header="true", inferSchema="true")
showDF(df_more_1000_distance)